In [2]:
from surya.tables import batch_table_recognition
from surya.detection import batch_text_detection
from surya.model.table_rec.model import load_model as load_model
from surya.model.table_rec.processor import load_processor
from surya.layout import batch_layout_detection
from surya.model.layout.model import load_model as load_layout_model
from surya.model.layout.processor import load_processor as load_layout_processor
from surya.model.detection.model import load_model as load_det_model, load_processor as load_det_processor
from surya.input.pdflines import get_table_blocks
from surya.postprocessing.util import rescale_bbox
from PIL import Image
import fitz

model = load_model()
processor = load_processor()



# get image of the first page
path = "C:/Users/aless/Downloads/Test.pdf"
doc = fitz.open(path)
page = doc[0]
#load page into PIL
image = page.get_pixmap(dpi=96)
image = Image.frombytes("RGB", [image.width, image.height], image.samples)
images = [image]
highres_image = page.get_pixmap(dpi=192)
highres_image = Image.frombytes("RGB", [highres_image.width, highres_image.height], highres_image.samples)
highres_images = [highres_image]


Loaded table recognition model vikp/surya_tablerec on device cpu with dtype torch.float32


([<PIL.Image.Image image mode=RGB size=794x1123>], ['Test'], [None])

In [ ]:

layout_model = load_layout_model()
layout_processor = load_layout_processor()

det_model = load_det_model()
det_processor = load_det_processor()

layout_predictions = batch_layout_detection(images, layout_model, layout_processor)
table_cells = []

table_imgs = []
table_counts = []
text_lines = [None]
for layout_pred, text_line, img, highres_img in zip(layout_predictions, text_lines, images, highres_images):
    # The table may already be cropped

    # The bbox for the entire table
    bbox = [l.bbox for l in layout_pred.bboxes if l.label == "Table"]
    # Number of tables per page
    table_counts.append(len(bbox))

    if len(bbox) == 0:
        continue

    page_table_imgs = []
    highres_bbox = []
    for bb in bbox:
        highres_bb = rescale_bbox(bb, img.size, highres_img.size)
        page_table_imgs.append(highres_img.crop(highres_bb))
        highres_bbox.append(highres_bb)

    table_imgs.extend(page_table_imgs)

    # The text cells inside each table
    table_blocks = get_table_blocks(highres_bbox, text_line, highres_img.size) if text_line is not None else None

    det_results = batch_text_detection(page_table_imgs, det_model, det_processor)
    cell_bboxes = [[{"bbox": tb.bbox, "text": None} for tb in det_result.bboxes] for det_result in det_results]
    table_cells.extend(cell_bboxes)


In [ ]:
table_preds = batch_table_recognition([image], table_cells, model, processor)